In [1]:
import pandas as pd

In [2]:
# raw_affective = pd.read_csv("/Users/calebjonesshibu/Desktop/tom/data/exp_2023_02_03_10/baseline_tasks/affective/individual_00104_1675449225.csv", sep=";")
# extracted_affective = pd.read_csv("/Users/calebjonesshibu/Desktop/Neurips/Affective_Task_Individual/exp_2023_02_03_10/104.csv", sep=",")
# # extracted_affective.drop("Unnamed: 0", axis=1, inplace=True)

In [3]:
# # raw_affective['arousal_score'].fillna(method='ffill', inplace=True)
# # raw_affective['valence_score'].fillna(method='ffill', inplace=True)
# # raw_affective['subject_id'].fillna(method='ffill', inplace=True)
# # Forward fill NaN values in all columns
# raw_affective.fillna(method='bfill', inplace=True)
# extracted_affective.fillna(method='bfill', inplace=True)

In [4]:
# raw_affective

In [5]:
# extracted_affective

In [6]:
# import pandas as pd

# # Assuming you have the DataFrames raw_affective and extracted_affective

# # Convert 'human_readable_time' column in extracted_affective to datetime type
# extracted_affective['human_readable_time'] = pd.to_datetime(extracted_affective['human_readable_time'])

# # Perform the merge based on closest matching time
# merged_df = pd.merge_asof(extracted_affective.sort_values('unix_time'),
#                           raw_affective.sort_values('time'),
#                           left_on='unix_time',
#                           right_on='time',
#                           direction='nearest')

In [49]:
data_1 = pd.read_csv('/Users/calebjonesshibu/Desktop/Neurips/Affective_Task_Individual_rating/exp_2023_02_21_14/109.csv')
data_1.drop(['unix_time', 'event_type_x', 'event_type_y', 'unix_time' ,'time', 'subject_id', 'image_path','human_readable_time_y', 'human_readable_time_x', 'monotonic_time'] , axis=1, inplace=True)

In [50]:
data_1.head()

,S1-D1_HbO,S1-D2_HbO,S2-D1_HbO,S2-D3_HbO,S3-D1_HbO,S3-D3_HbO,S3-D4_HbO,S4-D2_HbO,S4-D4_HbO,S4-D5_HbO,...,S5-D6_HbR,S6-D4_HbR,S6-D6_HbR,S6-D7_HbR,S7-D5_HbR,S7-D7_HbR,S8-D6_HbR,S8-D7_HbR,arousal_score,valence_score
0,0.471891,0.202912,-0.093997,0.134281,0.372591,0.322716,0.232153,-0.256723,0.219447,0.666889,...,-0.171817,-0.041364,-0.126358,-0.169818,0.146536,-0.133517,-0.148632,-0.006896,0.0,0.0
1,0.469043,0.198057,-0.108167,0.129077,0.367336,0.313904,0.238565,-0.258443,0.213415,0.635690,...,-0.171419,-0.040187,-0.127127,-0.166950,0.144688,-0.128004,-0.146233,-0.008951,0.0,0.0
2,0.465940,0.193808,-0.122093,0.122993,0.360953,0.304467,0.243875,-0.259318,0.207682,0.606112,...,-0.170762,-0.039057,-0.127535,-0.163997,0.143110,-0.122583,-0.143714,-0.010777,0.0,0.0
3,0.462609,0.190255,-0.135737,0.116034,0.353432,0.294504,0.248101,-0.259339,0.202299,0.578540,...,-0.169872,-0.037982,-0.127574,-0.161002,0.141842,-0.117328,-0.141122,-0.012323,0.0,0.0
4,0.459056,0.187457,-0.149122,0.108107,0.344656,0.284036,0.251216,-0.258564,0.197311,0.553323,...,-0.168782,-0.036988,-0.127237,-0.158030,0.140913,-0.112311,-0.138505,-0.013541,0.0,0.0


In [51]:
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [32]:
# Split data into train and test
train_data, test_data = train_test_split(data_1, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
train_data_normalized = scaler.fit_transform(train_data.drop(['arousal_score', 'valence_score', 'image_path'], axis=1))
test_data_normalized = scaler.transform(test_data.drop(['arousal_score', 'valence_score', 'image_path'], axis=1))

In [33]:
# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


# MLP regression task

In [34]:
# Create data loaders
batch_size = 32
train_dataset = CustomDataset(train_data_normalized, train_data[['arousal_score', 'valence_score']].values)
test_dataset = CustomDataset(test_data_normalized, test_data[['arousal_score', 'valence_score']].values)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
input_size = train_data_normalized.shape[1]
output_size = 2  # Predicting arousal_score and valence_score
hidden_size = 50

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.float()
        targets = targets.float()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/100, Loss: 0.02018042467534542
Epoch 20/100, Loss: 0.040841978043317795
Epoch 30/100, Loss: 0.0025714526418596506
Epoch 40/100, Loss: 0.019988734275102615
Epoch 50/100, Loss: 0.007794364355504513
Epoch 60/100, Loss: 0.005270054098218679
Epoch 70/100, Loss: 0.008183201774954796
Epoch 80/100, Loss: 0.009495115838944912
Epoch 90/100, Loss: 0.0024096586275845766
Epoch 100/100, Loss: 0.01624155417084694


In [81]:
model.eval()  # Switch to evaluation mode

with torch.no_grad():
    total_mae = 0
    total_mse = 0
    total_samples = 0

    for i, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.float()
        targets = targets.float()

        outputs = model(inputs)
        mae = torch.abs(outputs - targets).mean().item()  # MAE
        mse = ((outputs - targets) ** 2).mean().item()  # MSE

        total_mae += mae * inputs.size(0)  # Multiply by batch size
        total_mse += mse * inputs.size(0)  # Multiply by batch size
        total_samples += inputs.size(0)

    print('Test MAE: ', total_mae / total_samples)
    print('Test MSE: ', total_mse / total_samples)
    print('Test RMSE: ', (total_mse / total_samples) ** 0.5)


Test MAE:  0.05128848232328892
Test MSE:  0.01092806205386296
Test RMSE:  0.10453737156568918


In [82]:
# Get predictions
model.eval()  # Switch to evaluation mode

with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.float()
        targets = targets.float()

        # Make predictions
        outputs = model(inputs)
        arousal_predictions, valence_predictions = outputs[:, 0], outputs[:, 1]
        
        # Print the first few predictions
        for arousal_prediction, valence_prediction, target_arousal, target_valence in zip(arousal_predictions[:5], valence_predictions[:5], targets[:, 0][:5], targets[:, 1][:5]):
            print(f'Predicted arousal: {arousal_prediction:.2f}, Actual arousal: {target_arousal:.2f}')
            print(f'Predicted valence: {valence_prediction:.2f}, Actual valence: {target_valence:.2f}\n')

        # Break after the first batch to only print a few predictions
        break


Predicted arousal: -1.04, Actual arousal: -1.00
Predicted valence: 1.00, Actual valence: 1.00

Predicted arousal: -0.02, Actual arousal: 0.00
Predicted valence: -0.95, Actual valence: -1.00

Predicted arousal: 0.03, Actual arousal: 0.00
Predicted valence: -1.06, Actual valence: -1.00

Predicted arousal: 0.01, Actual arousal: 0.00
Predicted valence: -0.02, Actual valence: 0.00

Predicted arousal: 1.01, Actual arousal: 1.00
Predicted valence: 0.97, Actual valence: 1.00



# MLP multiclass classification problem

Transform the targets to be in the range [0, 4] instead of [-2, 2] because PyTorch's CrossEntropyLoss expects class labels to be in this format

In [35]:
# Create data loaders
batch_size = 32
train_dataset = CustomDataset(train_data_normalized, train_data[['arousal_score', 'valence_score']].values)
test_dataset = CustomDataset(test_data_normalized, test_data[['arousal_score', 'valence_score']].values)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
input_size = train_data_normalized.shape[1]
output_size = 2  # Predicting arousal_score and valence_score
hidden_size = 200

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net(input_size, hidden_size, 2)  # Output size is 2

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


# Update your targets to be within [0, 4]
train_dataset.targets = train_dataset.targets + 2
test_dataset.targets = test_dataset.targets + 2

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.float()
        targets = targets.float()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 10/1000, Loss: 2.9827427864074707
Epoch 20/1000, Loss: 2.7489967346191406
Epoch 30/1000, Loss: 2.6320855617523193
Epoch 40/1000, Loss: 2.7617275714874268
Epoch 50/1000, Loss: 2.4745981693267822
Epoch 60/1000, Loss: 2.7816989421844482
Epoch 70/1000, Loss: 2.669663667678833
Epoch 80/1000, Loss: 2.932523250579834
Epoch 90/1000, Loss: 2.806131362915039
Epoch 100/1000, Loss: 2.6243016719818115
Epoch 110/1000, Loss: 2.838048219680786
Epoch 120/1000, Loss: 2.987449884414673
Epoch 130/1000, Loss: 2.4545645713806152
Epoch 140/1000, Loss: 2.7423462867736816
Epoch 150/1000, Loss: 2.5729427337646484
Epoch 160/1000, Loss: 2.4795849323272705
Epoch 170/1000, Loss: 2.9867165088653564
Epoch 180/1000, Loss: 2.676955461502075
Epoch 190/1000, Loss: 2.951040267944336
Epoch 200/1000, Loss: 2.5577430725097656
Epoch 210/1000, Loss: 2.728386402130127
Epoch 220/1000, Loss: 3.013456106185913
Epoch 230/1000, Loss: 2.678410053253174
Epoch 240/1000, Loss: 2.750190258026123
Epoch 250/1000, Loss: 2.528984546661

In [21]:
model.eval()

correct_arousal = 0
correct_valence = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.float()
        targets = targets.long()  # CrossEntropyLoss expects targets to be of type long

        outputs = model(inputs)
        
        _, predicted = outputs.max(dim=1)
        
        predicted_arousal = predicted[0]
        predicted_valence = predicted[1]

        total += targets.size(0)
        correct_arousal += (predicted_arousal == targets[:, 0]).sum().item()
        correct_valence += (predicted_valence == targets[:, 1]).sum().item()

print('Accuracy of the network on the test data for arousal_score: %d %%' % (
    100 * correct_arousal / total))

print('Accuracy of the network on the test data for valence_score: %d %%' % (
    100 * correct_valence / total))


Accuracy of the network on the test data for arousal_score: 9 %
Accuracy of the network on the test data for valence_score: 10 %


# LSTM classification

In [53]:

# Preprocess data
features = data_1.iloc[:, :-2].values
arousal_score = LabelEncoder().fit_transform(data_1.iloc[:, -2] + 2)  # Mapping -2 -> 0, -1 -> 1, 0 -> 2, 1 -> 3, 2 -> 4
valence_score = LabelEncoder().fit_transform(data_1.iloc[:, -1] + 2)  # Same mapping for valence_score
targets = list(zip(arousal_score, valence_score))

# Split data
train_features, test_features, train_targets, test_targets = train_test_split(features, targets, test_size=0.2, random_state=42)

# Hyperparameters
input_size = features.shape[1]
hidden_size = 64
num_classes = 5  # Classes representing -2, -1, 0, 1, 2
num_epochs = 1000
batch_size = 32
learning_rate = 0.001

# Create DataLoaders
train_data = TensorDataset(torch.tensor(train_features).float(), torch.tensor(train_targets).long())
test_data = TensorDataset(torch.tensor(test_features).float(), torch.tensor(test_targets).long())
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Define model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Initialize model, loss and optimizer
model = LSTM(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.view(-1, 1, input_size)
        targets_arousal = targets[:, 0]
        targets_valence = targets[:, 1]

        outputs = model(inputs)
        loss_arousal = criterion(outputs, targets_arousal)
        loss_valence = criterion(outputs, targets_valence)

        loss = loss_arousal + loss_valence  # Total loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Testing
correct_arousal, correct_valence = 0, 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.view(-1, 1, input_size)
        targets_arousal = targets[:, 0]
        targets_valence = targets[:, 1]
        
        outputs = model(inputs)

        _, predicted_arousal = torch.max(outputs.data, 1)
        _, predicted_valence = torch.max(outputs.data, 1)

        total += targets.size(0)
        correct_arousal += (predicted_arousal == targets_arousal).sum().item()
        correct_valence += (predicted_valence == targets_valence).sum().item()

print(f'Accuracy of the network on the test data for arousal_score: {100 * correct_arousal / total} %')
print(f'Accuracy of the network on the test data for valence_score: {100 * correct_valence / total} %')

Epoch 10/1000, Loss: 1.6886212825775146
Epoch 20/1000, Loss: 1.2943766117095947
Epoch 30/1000, Loss: 1.550680160522461
Epoch 40/1000, Loss: 1.1466724872589111
Epoch 50/1000, Loss: 1.1815441846847534
Epoch 60/1000, Loss: 1.2847070693969727
Epoch 70/1000, Loss: 1.1381614208221436
Epoch 80/1000, Loss: 0.827776312828064
Epoch 90/1000, Loss: 1.1725575923919678
Epoch 100/1000, Loss: 1.1144821643829346
Epoch 110/1000, Loss: 0.9603128433227539
Epoch 120/1000, Loss: 1.1071457862854004
Epoch 130/1000, Loss: 1.2013905048370361
Epoch 140/1000, Loss: 1.1059744358062744
Epoch 150/1000, Loss: 0.9586115479469299
Epoch 160/1000, Loss: 1.0871164798736572
Epoch 170/1000, Loss: 1.203660249710083
Epoch 180/1000, Loss: 0.9663504362106323
Epoch 190/1000, Loss: 1.0540177822113037
Epoch 200/1000, Loss: 1.1633906364440918
Epoch 210/1000, Loss: 1.1072871685028076
Epoch 220/1000, Loss: 1.0720463991165161
Epoch 230/1000, Loss: 1.0737779140472412
Epoch 240/1000, Loss: 1.0027353763580322
Epoch 250/1000, Loss: 1.1597